In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [3]:
# Cargar los datos
df = pd.read_excel('C:/Users/Pedro/Desktop/DataScience-SoyHenry/AGSM--power bi/BASE PRUEBA BI.xlsx', sheet_name='DATOS')

# --- 1. Limpieza inicial ---


In [4]:

# Limpiar registros sin estado
df = df[df["Estado"].notnull()]

# Lista de estados que implican venta exitosa
estados_exitosos = [
    "abierto en solicitud de plan de ahorro",
    "abierto en preventa",
    "abierto en etapa de cierre de venta"
]

# Crear variable binaria objetivo
df["Estado_Binario"] = df["Estado"].str.lower().apply(lambda x: 1 if x in estados_exitosos else 0)


# --- 2. Crear variables derivadas ---

In [5]:
# Variable: cantidad de supervisores
df["Cantidad_Supervisores"] = df["Supervisores"].astype(str).apply(lambda x: len(x.split(",")))

# Variable: día de la semana de alta
df["Dia_Semana"] = pd.to_datetime(df["Fecha alta"], errors="coerce").dt.dayofweek

# Variable: días desde el alta
df["Días"] = (pd.to_datetime("today") - pd.to_datetime(df["Fecha alta"], errors="coerce")).dt.days


# --- 3. Seleccionar variables de entrada (features) ---

In [6]:
features = [
    "Días", "Cantidad_Supervisores", "Dia_Semana",  # numéricas
    "Origen", "Tipo de operación", "Rubro", "Categoría", "Tipo", "Tipo de Proceso Comercial"  # categóricas
]

# Convertir columnas categóricas a string para evitar errores en OneHotEncoder
for col in ["Origen", "Tipo de operación", "Rubro", "Categoría", "Tipo", "Tipo de Proceso Comercial"]:
    df[col] = df[col].astype(str)


X = df[features].copy()
y = df["Estado_Binario"]


# --- 4. Separar tipos de variables ---

In [7]:

# Definir tipos
numeric_features = ["Días", "Cantidad_Supervisores", "Dia_Semana"]
categorical_features = ["Origen", "Tipo de operación", "Rubro", "Categoría", "Tipo", "Tipo de Proceso Comercial"]
text_features = []  # Podés agregar texto si tenés

# Transformadores
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# ColumnTransformer
transformers = [
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
]
preprocessor = ColumnTransformer(transformers=transformers)


# --- 5. Entrenar el modelo ---

In [8]:

# Crear pipeline completo
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", GradientBoostingClassifier())
])

# Separar train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Entrenar
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Días',
                                                   'Cantidad_Supervisores',
                                                   'Dia_Semana']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Origen',
                                                   'Tipo de operación', 'Rubro',
                                                   'Categoría', 'Tipo',
                                                   'Tipo de Proceso '
                                                   'Comercial'])])),
                ('classifier', GradientBoostingClassifier())])

# --- 6. Evaluar ---

In [9]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

Accuracy: 0.9528567768383295
ROC AUC: 0.9309295729870826
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      3568
           1       0.88      0.53      0.66       335

    accuracy                           0.95      3903
   macro avg       0.92      0.76      0.82      3903
weighted avg       0.95      0.95      0.95      3903



# --- 7. Importancia de variables ---

In [10]:

# Extraer importancia de features categóricas codificadas
feature_names = clf.named_steps["preprocessor"].transformers_[0][2] + \
                list(clf.named_steps["preprocessor"].transformers_[1][1].named_steps["onehot"].get_feature_names_out(categorical_features))

importances = clf.named_steps["classifier"].feature_importances_
feature_importance = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

print("\nImportancia de características:")
print(feature_importance.head(20))


Importancia de características:
                             Feature  Importance
198                        Rubro_nan    0.203237
17                   Origen_Referido    0.167481
319                    Categoría_nan    0.158961
15              Origen_Proactivación    0.120055
337  Tipo de Proceso Comercial_Usado    0.062429
19                Origen_Sin Definir    0.056775
11           Origen_Llamada Recibida    0.047356
18            Origen_Salón de Ventas    0.046017
0                               Días    0.025348
43             Tipo de operación_nan    0.016406
336   Tipo de Proceso Comercial_Plan    0.015394
1              Cantidad_Supervisores    0.014004
4                Origen_Consulta WEB    0.010997
335   Tipo de Proceso Comercial_Otro    0.008084
334   Tipo de Proceso Comercial_Cero    0.007978
8              Origen_Instagram_Form    0.006729
41         Tipo de operación_CONTADO    0.004455
2                         Dia_Semana    0.003127
126             Rubro_administrativa

In [12]:
import joblib

# Guardar el modelo
joblib.dump(clf, "modelo_sin_smote.pkl")

# Guardar los datos de test
joblib.dump(X_test, "X_test_sin_smote.pkl")
joblib.dump(y_test, "y_test_sin_smote.pkl")

# Guardar importancia de variables
feature_importance.to_csv("importancia_modelo_sin_smote.csv", index=False)